In [1]:
import os
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
from tkinter import messagebox
from moviepy.editor import *
from threading import Thread

# Funktion zum Konvertieren von Videos
def convert_videos():
    def conversion_thread():
        input_option = input_option_var.get()
        input_path = input_entry.get()
        output_folder = output_folder_entry.get()
        resize_resolution = resize_resolution_entry.get()
        codec_option = codec_var.get()
        bitrate_option = bitrate_entry.get()

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        progress_bar.start()

        if input_option == "Folder":
            video_files = [filename for filename in os.listdir(input_path) if filename.endswith(".mp4")]
            total_videos = len(video_files)

            for i, filename in enumerate(video_files):
                input_file_path = os.path.join(input_path, filename)
                output_filename = os.path.splitext(filename)[0] + ".ogv"  # Ensure .ogv extension
                output_path = os.path.join(output_folder, output_filename)

                video_clip = VideoFileClip(input_file_path)

                # Überprüfe, ob die Auflösung geändert werden soll
                if resize_resolution:
                    width, height = map(int, resize_resolution.split('x'))
                    video_clip = video_clip.resize((width, height))

                # Specify the video codec as 'libtheora' and audio codec as 'libvorbis'
                video_clip.write_videofile(output_path, codec='libtheora', audio_codec='libvorbis')

                # Update progress bar
                progress = (i + 1) / total_videos * 100
                progress_bar["value"] = progress
                root.update_idletasks()

        elif input_option == "File":
            if input_path.endswith(".mp4"):
                output_filename = os.path.splitext(os.path.basename(input_path))[0] + ".ogv"
                output_path = os.path.join(output_folder, output_filename)

                video_clip = VideoFileClip(input_path)

                # Überprüfe, ob die Auflösung geändert werden soll
                if resize_resolution:
                    width, height = map(int, resize_resolution.split('x'))
                    video_clip = video_clip.resize((width, height))

                video_clip.write_videofile(output_path, codec=codec_option, bitrate=bitrate_option)
            
                
                # Update progress bar
                progress_bar["value"] = 100

        progress_bar.stop()
        result_label.config(text="Conversion completed!")

        if exit_on_done_var.get():
            root.quit()


    convert_thread = Thread(target=conversion_thread)
    convert_thread.start()

# Funktion zum Überprüfen, ob MoviePy installiert ist
def check_moviepy_installed():
    try:
        import moviepy
        messagebox.showinfo("Moviepy Installed", "Moviepy is installed!")
    except ImportError:
        messagebox.showerror("Moviepy Not Installed", "Moviepy is not installed!")

# Funktion zum Auswählen des Quellordners oder der Quelldatei
def choose_input():
    input_option = input_option_var.get()

    if input_option == "Folder":
        input_folder = filedialog.askdirectory()
        input_entry.delete(0, tk.END)
        input_entry.insert(0, input_folder)
    elif input_option == "File":
        input_file = filedialog.askopenfilename(filetypes=[("MP4 Files", "*.mp4")])
        input_entry.delete(0, tk.END)
        input_entry.insert(0, input_file)

# Funktion zum Auswählen des Ausgabeordners
def choose_output_folder():
    output_folder = filedialog.askdirectory()
    output_folder_entry.delete(0, tk.END)
    output_folder_entry.insert(0, output_folder)

# Fortschrittsbalken-Callback-Funktion
def progressbar_callback(current, total):
    def update_progress(percentage):
        progress_bar["value"] = percentage
        root.update_idletasks()

    def progress_fn(d):
        percentage = (current / total) * 100
        update_progress(percentage)

    return progress_fn

# GUI-Erstellung
root = tk.Tk()
root.title("Video Converter")

main_frame = ttk.Frame(root, padding=10)
main_frame.grid(column=0, row=0, sticky=(tk.W, tk.E, tk.N, tk.S))

input_option_var = tk.StringVar(value="Folder")
exit_on_done_var = tk.BooleanVar(value=True)
codec_var = tk.StringVar(value="libtheora")

input_option_label = ttk.Label(main_frame, text="Select input type:")
input_option_label.grid(column=0, row=0, columnspan=2, pady=5)

input_option_menu = ttk.OptionMenu(main_frame, input_option_var, "Folder", "Folder", "File")
input_option_menu.grid(column=2, row=0, columnspan=2, pady=5)

input_label = ttk.Label(main_frame, text="Input (Folder or File):")
input_label.grid(column=0, row=1, columnspan=2, pady=5)

input_entry = ttk.Entry(main_frame)
input_entry.grid(column=2, row=1, columnspan=2, pady=5)

input_button = ttk.Button(main_frame, text="Select", command=choose_input)
input_button.grid(column=4, row=1, pady=5)

output_label = ttk.Label(main_frame, text="Output folder for OGV files:")
output_label.grid(column=0, row=2, columnspan=2, pady=5)

output_folder_entry = ttk.Entry(main_frame)
output_folder_entry.grid(column=2, row=2, columnspan=2, pady=5)

output_button = ttk.Button(main_frame, text="Select", command=choose_output_folder)
output_button.grid(column=4, row=2, pady=5)

resize_resolution_label = ttk.Label(main_frame, text="Resize Resolution (Optional):")
resize_resolution_label.grid(column=0, row=3, columnspan=2, pady=5)

resize_resolution_entry = ttk.Entry(main_frame)
resize_resolution_entry.grid(column=2, row=3, columnspan=2, pady=5)

codec_label = ttk.Label(main_frame, text="Codec:")
codec_label.grid(column=0, row=4, columnspan=2, pady=5)

codec_menu = ttk.OptionMenu(main_frame, codec_var, "libtheora", "libtheora", "libx264")
codec_menu.grid(column=2, row=4, columnspan=2, pady=5)

bitrate_label = ttk.Label(main_frame, text="Bitrate (Optional):")
bitrate_label.grid(column=0, row=5, columnspan=2, pady=5)

bitrate_entry = ttk.Entry(main_frame)
bitrate_entry.grid(column=2, row=5, columnspan=2, pady=5)

convert_button = ttk.Button(main_frame, text="Convert Videos", command=convert_videos)
convert_button.grid(column=0, row=6, columnspan=5, pady=10)

check_moviepy_button = ttk.Button(main_frame, text="Check Moviepy", command=check_moviepy_installed)
check_moviepy_button.grid(column=0, row=7, columnspan=5, pady=5)

exit_on_done_checkbox = ttk.Checkbutton(main_frame, text="Exit when done", variable=exit_on_done_var)
exit_on_done_checkbox.grid(column=0, row=8, columnspan=5, pady=5)

progress_bar = ttk.Progressbar(main_frame, mode='determinate')
progress_bar.grid(column=0, row=9, columnspan=5, pady=10)

result_label = ttk.Label(main_frame, text="")
result_label.grid(column=0, row=10, columnspan=5, pady=10)

root.mainloop()

Moviepy - Building video C:/Users/Leonhard Tilly/Videos\test.ogv.
MoviePy - Writing audio in testTEMP_MPY_wvf_snd.ogg


MoviePy - Done.
Moviepy - Writing video C:/Users/Leonhard Tilly/Videos\test.ogv



t:   9%|▉         | 199/2172 [00:23<03:57,  8.32it/s, now=None]

t:  19%|█▊        | 407/2172 [00:48<03:46,  7.79it/s, now=None]